In [2]:
!git clone https://github.com/RaphaelEse/LawGorithmML.git

Cloning into 'LawGorithmML'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 686 (delta 3), reused 0 (delta 0), pack-reused 676 (from 1)
Receiving objects: 100% (686/686), 50.60 MiB | 8.17 MiB/s, done.
Resolving deltas: 100% (342/342), done.
Updating files: 100% (784/784), done.


In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch
from datasets import Dataset
import os
import glob
import pandas as pd
import json
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import concurrent.futures
import multiprocessing
from sklearn.utils import resample

In [5]:
# Define the path to the full-text bills
bills_path = "/content/LawGorithmML/citations/citations/sample_bills"

# Get all text files in the directory
bill_files = sorted(glob.glob(os.path.join(bills_path, "*.txt")))

bill_texts = {}
def load_file(file):
    with open(file, "r", encoding="utf-8") as f:
        return os.path.basename(file), f.read()

# Load 10 bills using multi-threading
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(load_file, bill_files)

bill_texts = dict(results)

print(f"Loaded {len(bill_texts)} bills.")

Loaded 108 bills.


In [6]:
json_path = "/content/LawGorithmML/labeled_citations_zero_shot/"
# Get all JSON files in the directory
# Load cleaned citation JSON files
json_files = sorted(os.listdir(json_path))

# Read and combine JSON files
cleaned_citations = []
for file in json_files:
    with open(os.path.join(json_path, file), "r", encoding="utf-8") as f:
        cleaned_citations.extend(json.load(f))

# Convert to DataFrame
df_citations = pd.DataFrame(cleaned_citations)

df_citations["normCite"] = df_citations["normCite"].astype(str)

# Filter out citations without `normCite`
df_citations = df_citations[df_citations["normCite"].notna()]
df_citations = df_citations[df_citations["normCite"].str.strip() != ""]
# Display dataset size
print(f"Loaded {len(json_files)} cleaned citation files with {len(df_citations)} citations.")
df_citations.head()


Loaded 77 cleaned citation files with 17405 citations.


,text,startPosition,endPosition,normCite,citeType,altCite,pinCiteStr,pageRangeStr,nodeId,section,sectionAndSubSection,isShortCite,chunk_id,high_confidence
13,Sec. 604,6331,6339,33 usc 1384,USC,33 usc 1384,,,0,,,True,0.0,Precedent
31,Sec. 104,9376,9384,42 usc 5304,USC,42 usc 5304,,,0,,,True,0.0,Amendment
32,Sec. 105,9480,9488,42 usc 5305,USC,42 usc 5305,,,0,,,True,0.0,Example
44,Sec. 204,10663,10671,7 usc 7508,USC,7 usc 7508,,,0,,,True,0.0,Amendment
48,Sec. 211,11303,11311,34 usc 60531,USC,34 usc 60531,,,0,,,True,0.0,Amendment


In [7]:
# Define context window size (adjust as needed)
CONTEXT_WINDOW = 200  # Number of characters before and after citation

def extract_context(args):
    """Extracts context from a bill given a citation."""
    citation_text, bill_texts = args  # Unpack arguments
    for bill_name, bill_text in bill_texts.items():
        match = re.search(re.escape(citation_text), bill_text)
        if match:
            start, end = match.start(), match.end()
            before = bill_text[max(0, start - CONTEXT_WINDOW): start].strip()
            after = bill_text[end: min(len(bill_text), end + CONTEXT_WINDOW)].strip()
            context = before + " " + citation_text + " " + after

            # Clean text
            context = re.sub(r"\s+", " ", context)  # Remove excessive spaces
            context = re.sub(r"[^\w\s.,;:?!'\"-]", "", context)  # Remove unwanted symbols

            return context if len(context) > 20 else None  # Ensure meaningful context
    return None  # No match found

# Apply multiprocessing
with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    df_citations["context"] = pool.map(extract_context, [(text, bill_texts) for text in df_citations["text"]])


# Drop rows where no context was found
df_citations = df_citations.dropna(subset=["context"])

# Extracted contexts sample
df_citations.head()

,text,startPosition,endPosition,normCite,citeType,altCite,pinCiteStr,pageRangeStr,nodeId,section,sectionAndSubSection,isShortCite,chunk_id,high_confidence,context
13,Sec. 604,6331,6339,33 usc 1384,USC,33 usc 1384,,,0,,,True,0.0,Precedent,ERS Sec. 601. Older Americans Tribal Advisory ...
31,Sec. 104,9376,9384,42 usc 5304,USC,42 usc 5304,,,0,,,True,0.0,Amendment,erage of home or community-based services unde...
32,Sec. 105,9480,9488,42 usc 5305,USC,42 usc 5305,,,0,,,True,0.0,Example,Medicaid eligibility for working adults with d...
44,Sec. 204,10663,10671,7 usc 7508,USC,7 usc 7508,,,0,,,True,0.0,Amendment,e hospitals. Sec. 202. Extension of the Medica...
48,Sec. 211,11303,11311,34 usc 60531,USC,34 usc 60531,,,0,,,True,0.0,Amendment,ealth flexibilities. Sec. 210. Requiring modif...


In [9]:
# Keep only necessary columns
df = df_citations[["text", "normCite", "citeType", "context", "high_confidence"]]

# Rename columns for clarity
df = df.rename(columns={"high_confidence": "label"})

# Encode labels as numbers (since models need numerical labels)
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

# Save label mapping for later decoding
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)

# Split into train/test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["context"], df["label"], test_size=0.2, random_state=42
)

# Convert into Pandas DataFrame format
train_df = pd.DataFrame({"text": train_texts, "label": train_labels})
test_df = pd.DataFrame({"text": test_texts, "label": test_labels})

# Show some training examples
print(f"Training Samples: {len(train_df)}, Test Samples: {len(test_df)}")
train_df.head(10)

Label Mapping: {'Amendment': np.int64(0), 'Authority': np.int64(1), 'Definition': np.int64(2), 'Example': np.int64(3), 'Exception': np.int64(4), 'Precedent': np.int64(5)}
Training Samples: 13322, Test Samples: 3331


,text,label
22794,States as directed in Senate Report 118193: Pr...,0
3261,d mission-related costs to expand capacity and...,3
9563,"id""H23975D78CC654E6BA19522B6E431C896"" qb-displ...",3
10373,"ovided, That such funds shall be for infrastru...",3
6523,"3"" qb-display-quote""yes"" g Definitions In this...",3
25057,13 Institution of higher education The term in...,0
8062,"g after the section heading for section 22401,...",5
19533,E to assist the Secretary in carrying out the ...,5
13274,LEADERS IN THE INDO-PACIFIC REGION Section 126...,3
1900,"rovided, That such funds shall be for infrastr...",2


In [10]:
# LegalBERT
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Convert Pandas DF to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize dataset
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Load pre-trained model for classification
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(label_encoder.classes_))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/13322 [00:00<?, ? examples/s]

Map:   0%|          | 0/3331 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f" Using device: {device}")

 Using device: cpu


In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [16]:
model.to(device)
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=7,
    weight_decay=0.05,
    fp16=True,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)
# Train the model
trainer.train()


NameError: name 'model' is not defined

In [19]:
# Save model and tokenizer
model.save_pretrained("./fine_tuned_legalbert")
tokenizer.save_pretrained("./fine_tuned_legalbert")
# Zip the model folder
!zip -r fine_tuned_legalbert.zip fine_tuned_legalbert

# Download to local computer
from google.colab import files
files.download("fine_tuned_legalbert.zip")


('./fine_tuned_legalbert/tokenizer_config.json',
 './fine_tuned_legalbert/special_tokens_map.json',
 './fine_tuned_legalbert/vocab.txt',
 './fine_tuned_legalbert/added_tokens.json',
 './fine_tuned_legalbert/tokenizer.json')

In [10]:
#Retreive and use model
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_legalbert")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_legalbert")

OSError: Incorrect path_or_model_id: './fine_tuned_legalbert'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [11]:
demo_samples = df_citations.sample(5)["context"].tolist()

In [13]:
#Legal roBERTa
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Convert Pandas DF to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize dataset
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Load pre-trained model for classification
rModel = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(label_encoder.classes_))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/13322 [00:00<?, ? examples/s]

Map:   0%|          | 0/3331 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
rModel.to(device)
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=7,
    weight_decay=0.05,
    fp16=True,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=rModel,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)
# Train the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: esedebrk20 (esedebrk20-virginia-tech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Save model and tokenizer
model.save_pretrained("./fine_tuned_roberta")
tokenizer.save_pretrained("./fine_tuned_roberta")
# Download to local computer
from google.colab import files
files.download("fine_tuned_roberta.zip")